# K-Means: Optimization Techniques

In this notebook, we discuss techniques for optimizing the performance of K-Means. More specifically, we address the following two questions?


- Question 1: How to avoid converging to a suboptimal solution?
- Question 2: How to accelerate the speed of convergence?


These questions are resolved by the following two variants of the K-Means algorithm, respectively.

- K-Means++: Uses a smarter initialization step to ensure that the selected centroids are distant from one another.
- Elkan's K-Means: Accelerates the speed by avoiding many unnecessary distance calculations.

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

## Synthetic Dataset

We will use Scikit-Learn's "make_blobs" function to generate isotropic Gaussian blobs for clustering. 

This function provides greater control regarding the centers and standard deviations of each cluster.
 

### Blob Centers and Standard Deviations

First define 5 blob centers by providing their coordinates.

Then, define the standard deviations of each blob.

In [2]:
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])

blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

## Load the Data


Note that "X" represents the generated 2D samples.

And "y" represents the integer labels for cluster membership of each sample.

Since we generate five clusters, the indices would be: 0, 1, 2, 3, 4

In [3]:
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

# Question 1: How to avoid converging to a suboptimal solution?


## Solution: K-Means++

K-Means++ is an important improvement to the K-Means algorithm that aims to solve the following problem:

        How to avoid converging to a suboptimal solution?

It was proposed in a 2006 paper by David Arthur and Sergei Vassilvitskii. 

Unlike K-Means that selects the centroids randomly, K-Means++ employs a smarter initialization step. It ensures that the selected centroids are distant from one another which resolves the suboptimal convergence issue.

This initialization step requires some additional computation. But this drastically reduces the number of times the algorithm needs to be run to find the optimal solution.

The K-Means++ initialization algorithm is as follows:

- Take one centroid $c_1$, chosen uniformly at random from the dataset.
- Take a new center $c_i$, choosing an instance $\mathbf{x}_i$ with probability: $D(\mathbf{x}_i)^2$ / $\sum\limits_{j=1}^{m}{D(\mathbf{x}_j)}^2$ where $D(\mathbf{x}_i)$ is the distance between the instance $\mathbf{x}_i$ and the closest centroid that was already chosen. This probability distribution ensures that instances that are further away from already chosen centroids are much more likely be selected as centroids.
- Repeat the previous step until all $k$ centroids have been chosen.

The rest of the K-Means++ algorithm is just regular K-Means. 

With this initialization, the K-Means algorithm is much less likely to converge to a suboptimal solution, so it is possible to reduce n_init considerably. 

Most of the time, this largely compensates for the additional complexity of the initialization process.

To set the initialization to K-Means++, simply set init="k-means++" (this the default setting).

In [4]:
good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])

kmeans = KMeans(n_clusters=5, init=good_init, n_init=1, random_state=42)
kmeans.fit(X)

print("Inertia: ", kmeans.inertia_)

Inertia:  211.5985372581684



# Question 2: How to accelerate the speed of convergence?

## Solution: Elkan's K-Means

Elkan's variant of K-Means accelerates the algorithm by avoiding many unnecessary distance calculations.

This is achieved by exploiting the triangle inequality (given three points A, B and C, the distance AC is always such that AC ≤ AB + BC) and by keeping track of lower and upper bounds for distances between instances and centroids.

To use Elkan's variant of K-Means, just set algorithm="elkan". 

Note that it does not support sparse data, so by default, Scikit-Learn uses "elkan" for dense data, and "full" (the regular K-Means algorithm) for sparse data.


To evaluate the performance of the Elkan's K-Means algorithm we will use the "timeit" function.

### %timeit

To fit a model multiple time and computing its mean & std, we will use the %timeit function

%timeit is a magic function, which can be used to time a particular piece of code (A single execution statement, or a single method).

When called as a program from the command line, the following form is used:

[-n N] [-r N] [statement ...]

-n N, --number=N
how many times to execute ‘statement’

-r N, --repeat=N
how many times to repeat the timer (default 3)

-s S, --setup=S
statement to be executed once initially (default pass)

In [5]:
print("Time Required by Elkan's K-Means:\n")
%timeit -n 50 KMeans(algorithm="elkan").fit(X)

Time Required by Elkan's K-Means:

89.2 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 50 loops each)


In [6]:
print("Time Required by Vanilla K-Means:\n")
%timeit -n 50 KMeans(algorithm="full").fit(X)

Time Required by Vanilla K-Means:

113 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 50 loops each)
